## STEP-*2*

In [1]:
!pip install simpy

## STEP-*3*

In [2]:
import simpy

def clock(env, name, tick):
    while True:
        print(f"{name} at time {env.now}")
        # This is where the magic happens: the simulation 'waits'
        yield env.timeout(tick)

# 1. Create the environment
env = simpy.Environment()

# 2. Add processes to the environment
env.process(clock(env, 'Fast Clock', 1.0))
env.process(clock(env, 'Slow Clock', 3.5))

# 3. Run it for 10 time units
env.run(until=10)

Fast Clock at time 0
Slow Clock at time 0
Fast Clock at time 1.0
Fast Clock at time 2.0
Fast Clock at time 3.0
Slow Clock at time 3.5
Fast Clock at time 4.0
Fast Clock at time 5.0
Fast Clock at time 6.0
Slow Clock at time 7.0
Fast Clock at time 7.0
Fast Clock at time 8.0
Fast Clock at time 9.0


## STEP-*4*

In [6]:
import simpy
import random
import pandas as pd
import numpy as np

def warehouse_process(env, num_robots, arrival_rate, process_time, wait_times):
    robot_resource = simpy.Resource(env, capacity=num_robots)
    while True:
        yield env.timeout(random.expovariate(1.0 / arrival_rate))
        arrival_time = env.now
        with robot_resource.request() as request:
            yield request
            # Adding 'Noise': real robots vary in speed every time
            actual_job_duration = random.gauss(process_time, 2.0)
            yield env.timeout(max(0.1, actual_job_duration))
            wait_times.append(env.now - arrival_time)

def run_simulation(n_robots, a_rate, p_time):
    env = simpy.Environment()
    wait_times = []
    env.process(warehouse_process(env, n_robots, a_rate, p_time, wait_times))
    env.run(until=480)
    # Adding a 'System Glitch' factor: 5% chance of a massive delay
    base_avg = np.mean(wait_times) if wait_times else 0
    if random.random() < 0.05: base_avg *= 1.5
    return base_avg

# Generate New Data
data_rows = []
for i in range(1000):
    n_robots = random.randint(1, 20)
    a_rate = random.uniform(2.0, 15.0)
    p_time = random.uniform(3.0, 12.0)
    avg_wait = run_simulation(n_robots, a_rate, p_time)
    data_rows.append({'num_robots': n_robots, 'arrival_rate': a_rate, 'process_time': p_time, 'avg_wait_time': avg_wait})

df = pd.DataFrame(data_rows)
df.to_csv('warehouse_data_v2.csv', index=False)
print("New, realistic dataset created!")

New, realistic dataset created!


# STEP-*5*

In [7]:

df = pd.read_csv('warehouse_data_v2.csv')

print(df.head())

print(df.describe())

   num_robots  arrival_rate  process_time  avg_wait_time
0          20      8.636156     11.128049      11.018588
1           6     13.364032      7.530694       7.809437
2          16     14.282148      5.010451       5.318448
3           4     11.767669      5.720350       5.688438
4          20      5.939002      7.971642       7.937993
        num_robots  arrival_rate  process_time  avg_wait_time
count  1000.000000   1000.000000   1000.000000    1000.000000
mean     10.203000      8.443807      7.455073       7.664041
std       5.660551      3.799545      2.663872       2.911378
min       1.000000      2.003561      3.000953       2.651470
25%       5.000000      5.225459      4.973460       5.227559
50%      10.000000      8.456888      7.593287       7.686797
75%      15.000000     11.718577      9.806853       9.962766
max      20.000000     14.968261     11.986841      18.237425


In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error, root_mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor

# 1. Load the dataset
df = pd.read_csv('warehouse_data_v2.csv')
X = df[['num_robots', 'arrival_rate', 'process_time']]
y = df['avg_wait_time']

# 2. Split into Training and Testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Initialize the models
models = {
    "Linear Regression": LinearRegression(),
    "Decision Tree": DecisionTreeRegressor(random_state=42),
    "Random Forest": RandomForestRegressor(n_estimators=100, random_state=42),
    "K-Nearest Neighbors": KNeighborsRegressor(),
    "Gradient Boosting": GradientBoostingRegressor(random_state=42),
    "XGBoost": XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
}

# 4. Train and Evaluate each model
results = []

for name, model in models.items():
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)

    # Calculate Metrics
    r2 = r2_score(y_test, predictions)
    mae = mean_absolute_error(y_test, predictions)
    rmse = root_mean_squared_error(y_test, predictions)

    results.append({
        "Model": name,
        "R2 Score": round(r2, 4),
        "MAE (mins)": round(mae, 2),
        "RMSE": round(rmse, 2)
    })

# 5. Display the Comparison Table
results_df = pd.DataFrame(results).sort_values(by="R2 Score", ascending=False)
print("--- ML Model Comparison Table ---")
print(results_df.to_string(index=False))

# 6. Report the Winner
winner = results_df.iloc[0]
print(f"\n🏆 THE BEST MODEL IS: {winner['Model']}")
print(f"It explains {winner['R2 Score']*100:.2f}% of the variance in wait times.")

--- ML Model Comparison Table ---
              Model  R2 Score  MAE (mins)  RMSE
  Linear Regression    0.9000        0.51  0.96
  Gradient Boosting    0.8899        0.54  1.01
      Random Forest    0.8775        0.61  1.06
K-Nearest Neighbors    0.8740        0.68  1.08
            XGBoost    0.8566        0.63  1.15
      Decision Tree    0.8021        0.71  1.35

🏆 THE BEST MODEL IS: Linear Regression
It explains 90.00% of the variance in wait times.


In [11]:
lr_model = models["Linear Regression"]
coefficients = pd.DataFrame({
    'Parameter': ['Num Robots', 'Arrival Rate', 'Process Time'],
    'Impact Score': lr_model.coef_
})

print("--- HOW THE AI THINKS ---")
print(coefficients)

--- HOW THE AI THINKS ---
      Parameter  Impact Score
0    Num Robots     -0.003856
1  Arrival Rate     -0.005281
2  Process Time      1.029777
